In [19]:
import tifffile
import python_test
import pandas as pd
import napari
import numpy as np
import skimage
from scipy import ndimage
from cellpose import models
import matplotlib.pyplot as plt
import os
from sklearn.metrics import f1_score
from sklearn.metrics import jaccard_score

In [10]:
viewer = napari.Viewer()

In [7]:
model_path = '/home/sim/OneDrive/Data/by_projects/gcamp/d/train_data/cellpose/models/cellpose_residual_on_style_on_concatenation_off_cellpose_2023_11_30_15_15_43.973221'
data = '/home/sim/OneDrive/Data/by_projects/gcamp/d/train_data/images/'
labels = '/home/sim/OneDrive/Data/by_projects/gcamp/d/train_data/labels/'
X = sorted(os.listdir(data))
X = [tifffile.imread(data + i) for i in X]
Y_true = sorted(os.listdir(labels))
Y_true = np.array([tifffile.imread(labels + i) for i in Y_true])

In [8]:
model = models.CellposeModel(gpu = True, pretrained_model=model_path)
masks, flows, styles = model.eval(X,
                                   diameter=30, channels=[0,0],
                                      flow_threshold=0.4, do_3D=False)
Y_pred = np.array(masks)

In [25]:
model_path = "./for-the-report-cyto2"
model = models.CellposeModel(gpu = True, pretrained_model=model_path)
masks, flows, styles = model.eval(X,
                                   diameter=30, channels=[0,0],
                                      flow_threshold=0.4, do_3D=False)
Y_pred2 = np.array(masks)

In [26]:
viewer.add_image(Y_pred2)

<Image layer 'Y_pred2' at 0x7f9a7714cb80>

In [11]:
viewer.add_image(np.array(X))
viewer.add_labels(Y_true)
viewer.add_labels(Y_pred)

<Labels layer 'Y_pred' at 0x7f9a14339d20>

In [40]:
def compute_iou_for_labels(true_labels, pred_labels):
    unique_true_labels = np.unique(true_labels)
    unique_pred_labels = np.unique(pred_labels)

    iou_scores = np.zeros((len(unique_true_labels), len(unique_pred_labels)))

    for i, true_label in enumerate(unique_true_labels):
        for j, pred_label in enumerate(unique_pred_labels):
            true_mask = (true_labels == true_label).astype(int)
            pred_mask = (pred_labels == pred_label).astype(int)

            intersection = np.sum(np.logical_and(true_mask, pred_mask))
            union = np.sum(np.logical_or(true_mask, pred_mask))

            iou = intersection / union if union != 0 else 0
            iou_scores[i, j] = iou

    return iou_scores

def compute_max_iou_for_each_label(iou_scores):
    max_iou_for_each_true_label = np.max(iou_scores, axis=1)
    return max_iou_for_each_true_label



def do(x, y):# Assuming true_labels and pred_labels are the ground truth and predicted labels, respectively
    iou_scores = compute_iou_for_labels(x, y)
    max_iou_for_each_true_label = compute_max_iou_for_each_label(iou_scores)

    average_iou = np.mean(max_iou_for_each_true_label)
    print("Average Max IoU:", average_iou)

for i in range(Y_true.shape[0]):
    print(i)
    do(Y_true[i], Y_pred2[i])
    do(Y_true[i], Y_pred[i])

0
Average Max IoU: 0.8511065329312785
Average Max IoU: 0.8351684951101499
1
Average Max IoU: 0.8157236100872273
Average Max IoU: 0.8663929893576096
2
Average Max IoU: 0.8523786661804879
Average Max IoU: 0.8423120447125138
3
Average Max IoU: 0.8405531061039849
Average Max IoU: 0.868154096776257
4
Average Max IoU: 0.8599511759360795
Average Max IoU: 0.891358458688232
5
Average Max IoU: 0.8522786194765284
Average Max IoU: 0.8805699658003252
6
Average Max IoU: 0.844901443355413
Average Max IoU: 0.8916395376694901
7
Average Max IoU: 0.7120006773527144
Average Max IoU: 0.8377059789659965
8
Average Max IoU: 0.7769391256147538
Average Max IoU: 0.8511781803905006
9
Average Max IoU: 0.8683734953124803
Average Max IoU: 0.9051106880715538
10
Average Max IoU: 0.8620049085395544
Average Max IoU: 0.8941204625074428
11
Average Max IoU: 0.7538380636775086
Average Max IoU: 0.8171326748240922
12
Average Max IoU: 0.7159911401192475
Average Max IoU: 0.8211247943726975
13
Average Max IoU: 0.8710132321817649